In [85]:
import pandas as pd
import os
import math
import numpy as np

In [365]:
o = pd.read_csv(os.path.join("data", "all_day_211118.csv"))
s = pd.read_csv(os.path.join("data", "all_day_211206.csv"))
t = pd.read_csv(os.path.join("data", "all_day_220106.csv"))
f = pd.read_csv(os.path.join("data", "all_day_220107.csv"))

In [366]:
o_t = [
"- 00:00 ~ 09:30 → 수면",
"- 09:30 ~ 11:40 → 준비",
"- 11:45 ~ 12:00 → 이동",
"- 12:00 ~ 12:35 → 지하철",
"- 12:35 ~ 12:40 → 이동",
"- 12:40 ~ 12:45 → 지하철", 
"- 12:45 ~ 12:50 → 이동", 
"- 12:50 ~ 13:00 → 지하철", 
"- 13:00 ~ 13:10 → 이동", 
"- 13:10 ~ 14:30 → 정적활동",
"- 14:30 ~ 14:45 → 이동",
"- 14:45 ~ 14:55 → 정적활동",
"- 14:55 ~ 15:00 → 이동",
"- 15:00 ~ 15:40 → 면접",
"- 15:40 ~ 16:10 → 이동",
"- 16:10 ~ 16:50 → 정적활동",
"- 16:50 ~ 17:10 → 이동",
"- 17:10 ~ 17:45 → 정적활동",
"- 17:45 ~ 17:55 → 이동",
"- 17:55 ~ 18:10 → 지하철",
"- 18:10 ~ 18:15 → 이동",
"- 18:15 ~ 18:25 → 지하철",
"- 18:25 ~ 18:30 → 이동",
"- 18:30 ~ 19:10 → 지하철",
"- 19:10 ~ 19:30 → 이동"
]

s_t = [
"- 00:00 ~ 08:30 → 수면",
"- 08:30 ~ 10:40 → 준비",
"- 10:45 ~ 11:00 → 이동",
"- 11:00 ~ 11:35 → 지하철",
"- 11:35 ~ 11:45 → 지하철",
"- 12:45 ~ 11:55 → 지하철", 
"- 11:55 ~ 12:00 → 이동",
"- 12:00 ~ 12:10 → 지하철",    
"- 12:10 ~ 12:20 → 이동",
"- 12:20 ~ 13:30 → 정적활동",
"- 13:30 ~ 13:45 → 이동",
"- 13:45 ~ 13:55 → 정적활동",
"- 13:55 ~ 14:00 → 이동",
"- 14:00 ~ 14:40 → 면접",
"- 14:40 ~ 14:55 → 이동",
"- 14:55 ~ 16:00 → 정적활동",
"- 16:00 ~ 16:10 → 이동",
"- 16:10 ~ 16:25 → 지하철",
"- 16:25 ~ 16:30 → 이동",
"- 16:30 ~ 16:40 → 지하철",
"- 16:40 ~ 16:45 → 이동",
"- 16:45 ~ 17:20 → 지하철",
"- 17:20 ~ 17:40 → 이동"
]

t_t = [
"- 00:00 ~ 09:00 → 수면",
"- 09:00 ~ 13:00 → 준비",
"- 13:00 ~ 13:10 → 이동",
"- 13:10 ~ 13:45 → 지하철",
"- 13:45 ~ 14:00 → 이동",
"- 14:00 ~ 14:30 → 정적활동",
"- 14:30 ~ 14:40 → 이동",
"- 14:40 ~ 16:15 → 정적활동",
"- 16:15 ~ 17:15 → 면접",
"- 17:15 ~ 17:55 → 정적활동",
"- 17:55 ~ 18:30 → 이동",
]

f_t = [
"- 00:00 ~ 07:30 → 수면",
"- 07:30 ~ 08:45 → 준비",
"- 08:45 ~ 09:00 → 이동",
"- 09:00 ~ 09:25 → 지하철",
"- 09:25 ~ 09:30 → 이동",
"- 09:30 ~ 09:45 → 이동",
"- 09:45 ~ 09:55 → 이동",
"- 09:55 ~ 10:05 → 정적활동",
"- 10:05 ~ 10:20 → 면접",
"- 10:20 ~ 10:30 → 이동",
"- 10:30 ~ 10:45 → 지하철",
"- 10:45 ~ 10:50 → 이동",
"- 10:50 ~ 11:20 → 지하철",
"- 11:20 ~ 11:35 → 이동",
]


In [367]:
def split_text(texts):
    text = [t.split() for t in texts]
    text = [t[1::2] for t in text]
    text = [[time_to_num(t[0]), time_to_num(t[1]) - 1, t[2]]for t in text]
    return text

In [368]:
def time_to_num(time):
    time = int(time[:2]) * 60 + int(time[-2:])
    return time

In [369]:
# 1차_n
o_t = split_text(o_t)
# 2차_n
s_t = split_text(s_t)
# 1차_k
t_t = split_text(t_t)
# 1차_m
f_t = split_text(f_t)

In [370]:
def separate_data(data, time, info):
    complete = {"수면":[], "준비":[], "이동":[], "지하철":[], "정적활동":[], "면접":[]}
    for t in time:
        start, end = t[0], t[1]
        label = t[2]
        sep = 5

        while start < end:
            _h = sorted(list(data["hearts"][start:start+sep]))
            _h_center = _h[sep//2]
            _h_avg = np.mean(np.array(_h))
            
            _s = sorted(list(data["steps"][start:start+sep]))
            _s_center = _s[sep//2]
            _s_avg = np.mean(np.array(_s))
            
            h = _h_center if _h_center < _h_avg else _h_avg
            s = _s_center if _s_center < _s_avg else _s_avg
            
            complete[label].append([h,s,info])
            start += sep
        
    return complete

In [371]:
fod = separate_data(o, o_t, "n1")
fsd = separate_data(s, s_t, "n2")
ftd = separate_data(t, t_t, "k1")
ffd = separate_data(f, f_t, "m1")

In [372]:
keys = ["수면", "준비", "이동", "지하철", "정적활동", "면접"]

In [373]:
total_data = {}
for k in keys:
    total_data[k] = [*fod[k], *fsd[k], *ftd[k], *ffd[k]]

In [374]:
dataDF = pd.DataFrame([[t[0], t[1], t[2], k] for k in keys for t in total_data[k]])

In [375]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [376]:
dataDF.describe()

,0,1
count,803.000000,803.000000
mean,74.513584,48.362985
std,17.712670,92.897285
min,54.750000,2.125000
25%,62.000000,10.589063
50%,66.490625,17.953125
75%,82.675781,30.750000
max,142.500000,978.500000


In [377]:
# min-max
def normalize(data):
    normalized = []
    for value in data:
        normalized_num = (value - min(data)) / (max(data) - min(data)) * 10
        normalized.append(normalized_num)
        
    return normalized

In [378]:
for i in range(len(dataDF.columns) - 2):
    dataDF[i] = normalize(dataDF[i])

In [379]:
dataDF.describe()

,0,1
count,803.000000,803.000000
mean,2.252260,0.473568
std,2.018538,0.951451
min,0.000000,0.000000
25%,0.826211,0.086689
50%,1.337963,0.162111
75%,3.182425,0.293176
max,10.000000,10.000000


In [382]:
nerveDF = dataDF[dataDF[3]=="면접"]

In [383]:
nerveData = nerveDF[[0,1]]
nerveLabel = nerveDF[3]

In [384]:
dataDF = dataDF[dataDF[3]!="면접"]

In [385]:
allData = dataDF[[0,1]]
allLabel = dataDF[3]

In [404]:
nerveDF.index = [i for i in range(31)]

In [411]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors = 25)
classifier.fit(allData, allLabel)

KNeighborsClassifier(n_neighbors=25)

In [412]:
classifier.predict(nerveData)

array(['정적활동', '수면', '수면', '수면', '수면', '수면', '수면', '수면', '이동', '수면', '수면',
       '수면', '수면', '수면', '수면', '준비', '정적활동', '정적활동', '정적활동', '정적활동',
       '정적활동', '준비', '준비', '준비', '정적활동', '정적활동', '정적활동', '정적활동', '정적활동',
       '정적활동', '준비'], dtype=object)

In [413]:
predicSR = pd.Series(classifier.predict(nerveData))
result4 = pd.concat([nerveDF,predicSR], axis=1)

In [414]:
result4

,0,1,2,3,0
0,5.199430,0.788631,n1,면접,정적활동
1,0.598291,0.638203,n1,면접,수면
2,1.413105,0.487774,n1,면접,수면
3,1.282051,0.305851,n1,면접,수면
4,0.914530,0.131865,n1,면접,수면
5,0.683761,0.089873,n1,면접,수면
6,0.598291,0.060172,n1,면접,수면
7,0.358974,0.060172,n1,면접,수면
8,9.253561,1.814620,n2,면접,이동
9,0.997151,0.181795,n2,면접,수면
